In [ ]:
# import Bio.SeqUtils
# import Bio.Data.IUPACData as IUPACData
# import itertools
# import logging
# logging.basicConfig(level=logging.INFO)
# import matplotlib
# import pandas as pd
# import pyrosetta
# import pyrosetta.distributed.io as io
# import pyrosetta.distributed.packed_pose as packed_pose
# import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
# import pyrosetta.distributed.tasks.score as score
# import seaborn

# from dask.distributed import Client, LocalCluster
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import itertools
from multiprocessing import Pool

import pyrosetta

pyrosetta.init()


core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: 2018.51.post.dev+654.v2019.01dev60566.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=680383466 seed_offset=0 real_seed=680383466
core.init.random: RandomGenerator:init: Normal mode, seed=680383466 RG_type=mt19937


In [2]:
# import pose from pdb

pdb = '1stn-relaxed.pdb'
relaxed_pose = pyrosetta.pose_from_pdb(pdb)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 636 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.974893 seconds.
core.import_pose.import_pose: File '1stn-relaxed.pdb' automatically determined to be of type PDB


In [14]:
# make backrub ensemble from WT

n_backrub = 50

backrub_protocol = pyrosetta.rosetta.protocols.backrub.BackrubProtocol()
backrub_pose_list = []

# for i in range(n_backrub):
#     pose = relaxed_pose.clone()
#     pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, "structure_num", i)
#     backrub_protocol.apply(pose)
#     backrub_pose_list.append(pose)

core.mm.MMBondAngleLibrary: MM bond angle sets added fully assigned: 603; wildcard: 0 and 1 virtual parameter.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.380838 seconds to load from binary


In [8]:
# read in mutations from Kellogg dataset

df = pd.read_csv('./kellogg.csv', comment = '#')
df['chain'], df['wt_res'], df['pdb_res_index'], df['mut_res'] = df['Mutations'].str.split(' ', 3).str
df.res_index = df.pdb_res_index.astype(int)
df = df.drop('Mutations', axis=1)
df.head()

/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res
0,1,107L,-0.53,0.435294117647,H,H,0.53,A,G,44,S
1,2,160L,-0.20,0.070796460177,H,H,0.2,A,A,120,M
2,3,161L,0.17,0.212389380531,H,H,-0.17,A,A,116,N
3,4,162L,-0.24,0.389380530973,H,H,0.24,A,A,122,Q
4,5,163L,-0.22,0.46017699115,T,O,0.22,A,A,123,Q


In [11]:
# pick mutations based on pdb

pdb_id = pdb[:4].upper()
pdb_ddgs = df[df.PDBFileID == pdb_id]
pdb_ddgs['res_index'] = pdb_ddgs.apply(lambda x: relaxed_pose.pdb_info().pdb2pose(x['chain'], int(x['pdb_res_index'])), axis=1)

print('PDB: {}'.format(pdb_id))
print('Max ddG = {}'.format(pdb_ddgs.DDG.max()))
print('Min ddG = {}'.format(pdb_ddgs.DDG.min()))

pdb_ddgs.head()


PDB: 1STN
Max ddG = 7.5
Min ddG = -1.25


/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res,res_index
581,706,1STN,5.2,0.0,H,H,5.2,A,N,100,A,95
582,707,1STN,5.1,0.0,H,H,5.1,A,N,100,G,95
583,558,1STN,1.9,0.229508196721,H,H,1.9,A,E,101,A,96
584,559,1STN,3.1,0.229508196721,H,H,3.1,A,E,101,G,96
585,515,1STN,1.9,0.150442477876,H,H,1.9,A,A,102,F,97


In [12]:
# prepare mutation arguments from dataframe

mutations = list(zip(pdb_ddgs.chain,
                     pdb_ddgs.wt_res,
                     pdb_ddgs.res_index,
                     pdb_ddgs.mut_res))

In [15]:
[(structure_num, chain, wt_res, res_index, mut_res)
                for structure_num, (chain, wt_res, res_index, mut_res)
                in itertools.product(range(1, n_backrub), mutations)]      

[(1, 'A', 'N', 95, 'A'),
 (1, 'A', 'N', 95, 'G'),
 (1, 'A', 'E', 96, 'A'),
 (1, 'A', 'E', 96, 'G'),
 (1, 'A', 'A', 97, 'F'),
 (1, 'A', 'A', 97, 'G'),
 (1, 'A', 'L', 98, 'A'),
 (1, 'A', 'L', 98, 'G'),
 (1, 'A', 'V', 99, 'A'),
 (1, 'A', 'V', 99, 'G'),
 (1, 'A', 'V', 99, 'S'),
 (1, 'A', 'V', 99, 'T'),
 (1, 'A', 'R', 100, 'A'),
 (1, 'A', 'R', 100, 'F'),
 (1, 'A', 'R', 100, 'G'),
 (1, 'A', 'Q', 101, 'A'),
 (1, 'A', 'Q', 101, 'G'),
 (1, 'A', 'L', 103, 'A'),
 (1, 'A', 'L', 103, 'G'),
 (1, 'A', 'A', 104, 'G'),
 (1, 'A', 'E', 5, 'A'),
 (1, 'A', 'E', 5, 'G'),
 (1, 'A', 'K', 105, 'A'),
 (1, 'A', 'K', 105, 'G'),
 (1, 'A', 'V', 106, 'A'),
 (1, 'A', 'V', 106, 'G'),
 (1, 'A', 'V', 106, 'S'),
 (1, 'A', 'V', 106, 'T'),
 (1, 'A', 'A', 107, 'F'),
 (1, 'A', 'A', 107, 'G'),
 (1, 'A', 'A', 107, 'V'),
 (1, 'A', 'Y', 108, 'A'),
 (1, 'A', 'Y', 108, 'F'),
 (1, 'A', 'Y', 108, 'G'),
 (1, 'A', 'Y', 108, 'L'),
 (1, 'A', 'V', 109, 'A'),
 (1, 'A', 'V', 109, 'G'),
 (1, 'A', 'V', 109, 'S'),
 (1, 'A', 'V', 109, 'T'),
 (

In [ ]:
# run mutate and minimize, using multiprocessing

with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:
        work = [(structure_num, chain, wt_res, res_index, mut_res)
                for structure_num, (chain, wt_res, res_index, mut_res)
                in itertools.product(range(1, n_backrub), mutations)]          
        logging.info("Now generating mutations and minimizing")
        all_structures = p.starmap(MUTATE_AND_MINIMIZE, work)
        
        
        

In [1]:
### Pseudo Code

""" 
pass the 50 structures into a "mutate" function, which makes 50 structure of the same mutation, starting from the
    50 WT backrubbed structures
 at this point, you should have 50 structures for each point mutation tested, plus the WT (50 structures)


for each mutation:
    for each backrub structure:
        minimize


now we have n_mutations objects with 50 poses, each pose can be scored

calculate a ddG of score per pose

calculate an average ddG of mutation compared to wt average ddG

store in a pandas dataframe

make a heatmap
        

"""

In [ ]:
def mutate_and_minimize(pose, residue_index):
    
    
    
    return pose
    